In [37]:
import pandas as pd
import glob
import re
import numpy as np

The GHCN files are in a fixed-width format. Each row represents a month of data for a variable recorded at a station. The first four columns contain the station id, year, month, and element being measured. The remaining columns are the daily value (VALUE1) and 3 daily flags (MFLAG1, QFLAG1, SFLAG1). Each day has these four columns with the number being the day of the month.

The tedious part of this is generating the list of column widths so the data will be separated correctly, and generating the list of column names. The pattern for both is specified in the data readme.

Code to generate widths argument:

In [38]:
# initial identifier columns
widths = [11, 4, 2, 4]

# generate column widths for 31 days
for i in range(1, 32):
    exend_widths = [5, 1, 1, 1]
    widths.extend(exend_widths)

Code to generate names argument:

In [39]:
# initial identifier columns
names = ['id', 'year', 'month', 'element']

# generate names for daily columns
for i in range(1, 32):
    extend_names = [f'value{i}', f'mflag{i}', f'qflag{i}', f'sflag{i}']
    names.extend(extend_names)

In [40]:
test = pd.read_fwf('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/all/USC00401553.dly', widths = widths, header = None, names = names)

In [41]:
test.head()

,id,year,month,element,value1,mflag1,qflag1,sflag1,value2,mflag2,...,qflag29,sflag29,value30,mflag30,qflag30,sflag30,value31,mflag31,qflag31,sflag31
0,USC00401553,1897,3,TMAX,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
1,USC00401553,1897,3,TMIN,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
2,USC00401553,1897,3,PRCP,0,P,NaN,6.0,0,P,...,NaN,6.0,198,NaN,NaN,6.0,203,NaN,NaN,6.0
3,USC00401553,1897,3,SNOW,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
4,USC00401553,1897,4,TMAX,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN


Process all selected stations into a dataframe:

In [42]:
station_list = ['USC00401553', 'USC00401663', 'USC00403280', 'USC00404876',
                'USC00405061', 'USC00405108', 'USC00405349', 'USC00406371',
                'USC00406427', 'USC00406435', 'USC00406454', 'USC00406803',
                'USC00408414', 'USC00408562', 'USC00409907', 'USW00013827',
                'USW00013897']

# empty list to store dataframes
df_list = []

# loop over the station list to pull data and create dataframes
for station in station_list:
    url = f'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/all/{station}.dly'
    df_list.append(pd.read_fwf(url, widths = widths, header = None, names = names))

# concatenate to single dataframe
weather = pd.concat(df_list, ignore_index = True)

In [43]:
weather.tail()

,id,year,month,element,value1,mflag1,qflag1,sflag1,value2,mflag2,...,qflag29,sflag29,value30,mflag30,qflag30,sflag30,value31,mflag31,qflag31,sflag31
79295,USW00013897,2024,1,WDF5,300,NaN,NaN,W,290,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
79296,USW00013897,2024,1,WSF2,63,NaN,NaN,W,36,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
79297,USW00013897,2024,1,WSF5,134,NaN,NaN,W,63,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
79298,USW00013897,2024,1,WT01,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
79299,USW00013897,2024,1,WT08,-9999,NaN,NaN,NaN,-9999,NaN,...,NaN,NaN,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN


Melt and pivot dataframe so that there is a row per station per day:

In [44]:
# columns for melting
id_vars = ['id', 'year', 'month', 'element']
value_vars = weather.columns[4:-1]
var_name = 'day'
value_name = 'day_value'

weather_melt = pd.melt(weather, 
                       id_vars = id_vars,
                       value_vars = value_vars,
                       var_name = var_name,
                       value_name = value_name)

In [45]:
weather_melt.head()

,id,year,month,element,day,day_value
0,USC00401553,1897,3,TMAX,value1,-9999
1,USC00401553,1897,3,TMIN,value1,-9999
2,USC00401553,1897,3,PRCP,value1,0
3,USC00401553,1897,3,SNOW,value1,-9999
4,USC00401553,1897,4,TMAX,value1,-9999


Drop 'flag' rows:

In [46]:
weather_melt = weather_melt[~weather_melt['day'].str.contains('flag')]

Create date column from year, month, and extracted number from day column:

In [47]:
# day_num column from day
weather_melt['day_num'] = weather_melt['day'].str.extract('(\d+)')[0].str.zfill(2)

In [48]:
# same for month_num
weather_melt['month_num'] = weather_melt['month'].astype(str).str.zfill(2)

In [49]:
# concatenate into date column
weather_melt['date'] =  (weather_melt['year'].astype(str) + '-' + 
                         weather_melt['month_num'].astype(str) + '-' + 
                         weather_melt['day_num'].astype(str))

In [50]:
weather_melt.head()

,id,year,month,element,day,day_value,day_num,month_num,date
0,USC00401553,1897,3,TMAX,value1,-9999,01,03,1897-03-01
1,USC00401553,1897,3,TMIN,value1,-9999,01,03,1897-03-01
2,USC00401553,1897,3,PRCP,value1,0,01,03,1897-03-01
3,USC00401553,1897,3,SNOW,value1,-9999,01,03,1897-03-01
4,USC00401553,1897,4,TMAX,value1,-9999,01,04,1897-04-01


Drop unneeded columns:

In [51]:
weather_melt = weather_melt.drop(['day', 'day_num', 'month_num'], axis = 1)

In [52]:
weather_melt.head()

,id,year,month,element,day_value,date
0,USC00401553,1897,3,TMAX,-9999,1897-03-01
1,USC00401553,1897,3,TMIN,-9999,1897-03-01
2,USC00401553,1897,3,PRCP,0,1897-03-01
3,USC00401553,1897,3,SNOW,-9999,1897-03-01
4,USC00401553,1897,4,TMAX,-9999,1897-04-01


Replace missing observation value (-9999) with null. (Was previously dropping entirely but doing it this way will maintain a record for each day for each station instead of having days missing entirely.)

In [53]:
weather_melt['day_value'] = weather_melt['day_value'].replace(-9999, np.NaN)

In [54]:
weather_melt.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2458300 entries, 0 to 9595299
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   id         2458300 non-null  object 
 1   year       2458300 non-null  int64  
 2   month      2458300 non-null  int64  
 3   element    2458300 non-null  object 
 4   day_value  2111005 non-null  float64
 5   date       2458300 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 131.3+ MB


In [55]:
# core element measurements
element_keep = ['PRCP', 'TMIN', 'TMAX', 'SNOW', 'SNWD']

Keep only rows with elements of interest:

In [56]:
weather_melt = weather_melt[weather_melt['element'].isin(element_keep)]

In [57]:
weather_melt.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1522255 entries, 0 to 9595290
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   id         1522255 non-null  object 
 1   year       1522255 non-null  int64  
 2   month      1522255 non-null  int64  
 3   element    1522255 non-null  object 
 4   day_value  1444782 non-null  float64
 5   date       1522255 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 81.3+ MB


In [58]:
weather_pivot =  weather_melt.pivot_table('day_value', index = ['id', 'year', 'month', 'date'], columns = 'element').reset_index()

In [59]:
weather_pivot.head()

element,id,year,month,date,PRCP,SNOW,SNWD,TMAX,TMIN
0,USC00401553,1897,3,1897-03-01,0.0,NaN,NaN,NaN,NaN
1,USC00401553,1897,3,1897-03-02,0.0,NaN,NaN,NaN,NaN
2,USC00401553,1897,3,1897-03-03,117.0,NaN,NaN,NaN,NaN
3,USC00401553,1897,3,1897-03-04,0.0,NaN,NaN,NaN,NaN
4,USC00401553,1897,3,1897-03-05,109.0,NaN,NaN,NaN,NaN


In [60]:
weather_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360368 entries, 0 to 360367
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      360368 non-null  object 
 1   year    360368 non-null  int64  
 2   month   360368 non-null  int64  
 3   date    360368 non-null  object 
 4   PRCP    357187 non-null  float64
 5   SNOW    277372 non-null  float64
 6   SNWD    248533 non-null  float64
 7   TMAX    280841 non-null  float64
 8   TMIN    280849 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 24.7+ MB


In [61]:
weather_pivot.groupby(['id', 'year']).count().reset_index().sort_values('year').head(30)

element,id,year,month,date,PRCP,SNOW,SNWD,TMAX,TMIN
467,USC00406371,1890,31,31,31,0,0,1,1
468,USC00406371,1891,101,101,101,0,0,29,0
104,USC00403280,1891,153,153,153,2,0,0,0
105,USC00403280,1893,365,365,362,93,0,346,346
469,USC00406371,1893,342,342,319,121,0,87,87
470,USC00406371,1894,279,279,233,35,0,183,177
106,USC00403280,1894,304,304,304,122,0,295,295
471,USC00406371,1895,365,365,324,123,0,360,362
107,USC00403280,1895,303,303,302,151,0,303,303
472,USC00406371,1896,359,359,315,91,0,354,352


Thoughts on handling stations with spotty data:

Part of the benefit of using stations in relatively close proximity is to help fill in for data that might be missing at one station or another. I also only chose stations that had a reasonably long history and also high percent data completion. The real question might be to decide when to make the early cut off year-wise. Two stations have data starting in the 1890s but the records for both are spotty from the early years. It looks like 1898 is when those two stations become more reliable, and a third starts gathering data in that year and becomes complete in 1899 so 1899 or 1900 seems like a good starting point.

For handling mostly complete stations with a measurement missing here and there: I could take an average of the day before and the day after (at least with temperature) or just not consider nulls at all since I'll have data from several stations.

I'll need to do more detailed analysis to see if the snow data is complete enough to be useful. It is one of the core observations but there will necessarily be less data than temperature or general precipitation. My assumption is that this measurement will be more accurate for later time periods while precipitation and temperature should be fairly reliable even early in the time period covered by this data.

In [62]:
year_min = 1899
year_max = 2024

weather_pivot = weather_pivot[(weather_pivot['year'] >= year_min) & (weather_pivot['year'] < year_max)]

Converting measurements to standard:
- PRCP from tenths of mm to cm (inches might be better since it's standard in the US)
- SNOW and SNWD from mm to cm (or inches)
- TMAX and TMIN from tenths of degrees C to C (or F)

I'm leaning toward converting to US units just because it will make for better communication of data with the expected audience.

In [63]:
# PRCP in inches

weather_pivot['prcp_inches'] = weather_pivot['PRCP'].apply(lambda x : x / 254)

Interesting data quality issue:

There are ~40 rows where the max and min temperatures look like theye were reversed in the original data (out of ~360K rows). When converting the temps to F I'm going to address this by taking whichever is the higher value from the two columns as the max and the lower value as the min. There are approximately the same number of rows where the values are equal, so the conditional function will need to take that into account.

In [64]:
# function to convert tenths of degrees Celcius to Fahrenheit
def convert_to_f(row):
    if row['TMAX'] >= row['TMIN']:
        return ((row['TMAX'] / 10) * (9 / 5)) + 32, ((row['TMIN'] / 10) * (9 / 5)) + 32
    else:
        return ((row['TMIN'] / 10) * (9 / 5)) + 32, ((row['TMAX'] / 10) * (9 / 5)) + 32

# apply function across the dataframe
weather_pivot[['tmax_f', 'tmin_f']] = weather_pivot.apply(convert_to_f, 
                                                          axis=1, 
                                                          result_type='expand')

In [65]:
weather_pivot.head()

element,id,year,month,date,PRCP,SNOW,SNWD,TMAX,TMIN,prcp_inches,tmax_f,tmin_f
459,USC00401553,1899,1,1899-01-01,0.0,0.0,25.0,-33.0,-172.0,0.000000,26.06,1.04
460,USC00401553,1899,1,1899-01-02,0.0,0.0,25.0,44.0,-150.0,0.000000,39.92,5.00
461,USC00401553,1899,1,1899-01-03,0.0,0.0,NaN,78.0,17.0,0.000000,46.04,35.06
462,USC00401553,1899,1,1899-01-04,508.0,0.0,NaN,178.0,78.0,2.000000,64.04,46.04
463,USC00401553,1899,1,1899-01-05,64.0,0.0,NaN,133.0,0.0,0.251969,55.94,32.00


With the data mostly clean, I want to bring in station metadata:

In [66]:
station_metadata = pd.read_fwf('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt', 
                               names = ['ID', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME',
                                        'GSN FLAG', 'HCN/CRN FLAG'])

In [67]:

station_metadata.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP,NaN,GSN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL,NaN,NaN,41217.0


In [68]:
# filter to selected stations
station_metadata = station_metadata[station_metadata['ID'].isin(station_list)]

In [69]:
# drop unneeded columns from station data before merge

station_metadata = station_metadata.drop(['ELEVATION', 'NAME', 'GSN FLAG', 'HCN/CRN FLAG'], axis = 1).\
    rename({'ID': 'id', 'LATITUDE': 'lat', 'LONGITUDE': 'long', 'STATE': 'station_name'}, axis = 1)

In [70]:
weather_stations = pd.merge(weather_pivot, station_metadata, on = 'id')

In [71]:
weather_stations.head()

,id,year,month,date,PRCP,SNOW,SNWD,TMAX,TMIN,prcp_inches,tmax_f,tmin_f,lat,long,station_name
0,USC00401553,1899,1,1899-01-01,0.0,0.0,25.0,-33.0,-172.0,0.000000,26.06,1.04,36.55,-86.75,CEDAR HILL 1
1,USC00401553,1899,1,1899-01-02,0.0,0.0,25.0,44.0,-150.0,0.000000,39.92,5.00,36.55,-86.75,CEDAR HILL 1
2,USC00401553,1899,1,1899-01-03,0.0,0.0,NaN,78.0,17.0,0.000000,46.04,35.06,36.55,-86.75,CEDAR HILL 1
3,USC00401553,1899,1,1899-01-04,508.0,0.0,NaN,178.0,78.0,2.000000,64.04,46.04,36.55,-86.75,CEDAR HILL 1
4,USC00401553,1899,1,1899-01-05,64.0,0.0,NaN,133.0,0.0,0.251969,55.94,32.00,36.55,-86.75,CEDAR HILL 1


In [72]:
weather_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355611 entries, 0 to 355610
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            355611 non-null  object 
 1   year          355611 non-null  int64  
 2   month         355611 non-null  int64  
 3   date          355611 non-null  object 
 4   PRCP          352712 non-null  float64
 5   SNOW          276036 non-null  float64
 6   SNWD          248506 non-null  float64
 7   TMAX          276885 non-null  float64
 8   TMIN          276928 non-null  float64
 9   prcp_inches   352712 non-null  float64
 10  tmax_f        276928 non-null  float64
 11  tmin_f        276885 non-null  float64
 12  lat           355611 non-null  float64
 13  long          355611 non-null  float64
 14  station_name  355611 non-null  object 
dtypes: float64(10), int64(2), object(3)
memory usage: 43.4+ MB


In [75]:
weather_stations = weather_stations.drop(['year', 'month', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'lat', 'long'], axis = 1)

Add day of year column:

In [80]:
weather_stations['day_of_year'] = pd.to_datetime(weather_stations['date']).dt.dayofyear

In [81]:
weather_stations.to_csv('../data/weather.csv', index = False)